In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
data = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset_filtered.csv')

In [3]:
data['date_field'] = pd.to_datetime(data['date_field'])

In [5]:
data = data.set_index('date_field')


In [6]:
grouped = data.groupby('regional_master')['sales'].resample('W').sum()

In [25]:
def fit_model(group):
    model = ExponentialSmoothing(group, seasonal_periods=52)
    fitted_model = model.fit()
    forecast = fitted_model.forecast(52)  # Forecast for the next 52 weeks
    return forecast


In [26]:
forecasts = []
for rm, group in grouped.groupby(level=0):
    forecast_data = fit_model(group)
    forecast_df = pd.DataFrame({'regional_master': rm, 'forecast': forecast_data})
    forecasts.append(forecast_df)

forecasts = pd.concat(forecasts, ignore_index=True)


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  retur

In [27]:
forecasts

,regional_master,forecast
0,50000001,336183.587829
1,50000001,336183.587829
2,50000001,336183.587829
3,50000001,336183.587829
4,50000001,336183.587829
...,...,...
2699,50001136,56687.644253
2700,50001136,56687.644253
2701,50001136,56687.644253
2702,50001136,56687.644253


In [28]:
def smape(actual, forecast):
    """
    Calculate the Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    n = len(actual)
    combined = np.abs(actual) + np.abs(forecast)
    smape_values = np.abs(actual - forecast) / combined * 200 / n
    return np.mean(smape_values)


In [29]:
smape_scores = []
for rm, group in forecasts.groupby('regional_master'):
    actual = data.loc[data['regional_master'] == rm, 'sales'].values[-52:]
    forecast_values = group['forecast'].values
    smape_score = smape(actual, forecast_values)
    smape_scores.append((rm, smape_score))

In [30]:
print(smape_scores)

[(50000001, 0.35364589056450646), (50000002, 2.154508849752874), (50000004, 3.846153846153846), (50000006, 0.961748201370978), (50000007, 0.4636449945503443), (50000009, 0.8339409595009081), (50000010, 3.8375182067101035), (50000011, 0.7853646603692673), (50000020, 3.84615374508703), (50000022, 0.7186414623239958), (50000024, 0.3480709885215275), (50000027, 0.2530186938549306), (50000029, 0.98420804368104), (50000031, 0.2286795363882031), (50000033, 0.27151913314478077), (50000035, 0.8903881862301163), (50000041, 1.8592882225871372), (50000047, 1.3584481960405848), (50000051, 3.846153846153846), (50000053, 0.49564888866280843), (50000061, 0.3293917158159217), (50000065, 0.5867854824744602), (50000069, 0.5142423813952243), (50000071, 1.0700163083268255), (50000073, 0.5763070639577518), (50000074, 0.6839810967304625), (50000078, 0.41428666056888325), (50000152, 0.5148099145728728), (50000183, 0.42957941305871117), (50000274, 0.3988391465803237), (50000280, 1.636649846235423), (50000288, 

In [22]:
def mape(actual, forecast):
    """
    Calculate the Mean Absolute Percentage Error (MAPE)
    """
    actual, forecast = np.array(actual), np.array(forecast)
    mape_values = np.abs((actual - forecast) / actual) * 100
    return np.mean(mape_values)
